In [ ]:
# ==============================================================================
# NLP-SNN BENCHMARKING RUNNER
# ==============================================================================

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time

# IMPORT FUNCTIONS
# This executes the functions notebook to load the classes and logic.
%run nlp_snn_benchmarking_functions_50.py

# ==============================================================================
# CONFIGURATION
# ==============================================================================

# 1. DATASETS (3 diverse NLP tasks)
# - IMDB: Long text, Binary sentiment
# - AG News: Medium text, 4-class Topic
# - TREC: Short text, 6-class Question Type
datasets = ['imdb', 'ag_news', 'trec']

# 2. ENCODERS (Simple to Advanced)
# - DAN: Baseline (Averaging)
# - CNN: Structural (Conv1D)
# - LSTM: Sequential
# - BiLSTM_Attn: Advanced Sequential with Attention
# - Transformer: State-of-the-Art
encoders = ['DAN', 'CNN', 'LSTM', 'BiLSTM_Attn', 'Transformer']

# 3. SNN CONFIGS
# - 1SADP: One hidden layer
# - 2SADP: Two hidden layers (Deep SNN)
architectures = ['1SADP', '2SADP']
time_steps_list = [25, 50] # Simulation length (longer = more precise rate coding)

# Experiment Settings
# Lower epochs for demonstration. Increase for production (e.g., Encoder=10, SNN=20)
EPOCHS_ENCODER = 3
EPOCHS_SNN = 5

# ==============================================================================
# EXECUTION LOOP
# ==============================================================================

results_list = []

print("################################################################################")
print("STARTING NLP-SNN BENCHMARKING PIPELINE")
print("################################################################################")

for data_name in datasets:
    print(f"\n>>> PROCESSING DATASET: {data_name.upper()} <<<")

    for enc_name in encoders:
        print(f"\n   --- Encoder: {enc_name} ---")

        for arch in architectures:
            for t_steps in time_steps_list:

                print(f"      [Running] Arch: {arch} | T: {t_steps}")

                try:
                    # Run the full pipeline function defined in functions file
                    res = train_snn_pipeline(
                        dataset_name=data_name,
                        encoder_name=enc_name,
                        architecture=arch,
                        t_steps=t_steps,
                        epochs=EPOCHS_SNN,
                        encoder_epochs=EPOCHS_ENCODER
                    )

                    results_list.append(res)

                except Exception as e:
                    print(f"      [ERROR] Failed on {data_name}-{enc_name}-{arch}: {e}")
                    import traceback
                    traceback.print_exc()

# ==============================================================================
# RESULTS AGGREGATION
# ==============================================================================

df_results = pd.DataFrame(results_list)
print("\nBenchmark Complete!")
print(df_results)

# Save Raw Data
df_results.to_csv("nlp_snn_benchmark_results.csv", index=False)

# ==============================================================================
# VISUALIZATION
# ==============================================================================

if not df_results.empty:
    sns.set_theme(style="whitegrid")

    # 1. Encoder Performance Comparison (Aggregated over Archs)
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_results, x='encoder', y='test_acc', hue='dataset', palette='viridis')
    plt.title("Impact of NLP Encoder on SNN Accuracy", fontsize=16, fontweight='bold')
    plt.ylabel("Test Accuracy")
    plt.xlabel("Encoder Architecture")
    plt.legend(title='Dataset', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.savefig("encoder_comparison.png")
    plt.show()

    # 2. SNN Architecture Comparison (1SADP vs 2SADP)
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df_results, x='architecture', y='test_acc', hue='encoder', palette='Set2')
    plt.title("1-Hidden vs 2-Hidden Layers SNN Performance", fontsize=16, fontweight='bold')
    plt.ylabel("Test Accuracy")
    plt.savefig("arch_comparison.png")
    plt.show()

    # 3. Time Steps Impact
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df_results, x='timesteps', y='test_acc', hue='encoder', marker='o')
    plt.title("Effect of Simulation Time (T) on Accuracy", fontsize=16, fontweight='bold')
    plt.ylabel("Test Accuracy")
    plt.xlabel("Simulation Steps (T)")
    plt.xticks(time_steps_list)
    plt.savefig("timesteps_impact.png")
    plt.show()

    # 4. Training Time Analysis
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df_results, x='encoder', y='snn_train_time', hue='architecture', palette='Reds')
    plt.title("SNN Training Time by Encoder Input Complexity", fontsize=16, fontweight='bold')
    plt.ylabel("Training Time (seconds)")
    plt.savefig("training_time.png")
    plt.show()
else:
    print("No results to plot.")

################################################################################
STARTING NLP-SNN BENCHMARKING PIPELINE
################################################################################

>>> PROCESSING DATASET: IMDB <<<

   --- Encoder: DAN ---
      [Running] Arch: 1SADP | T: 25

[Data] Loading imdb...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[Data] imdb ready. Train shape: (25000, 100), Classes: 2

[Encoder] Initializing DAN...
[Encoder] Pre-training DAN on imdb for 3 epochs...
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.7299 - loss: 0.5301 - val_accuracy: 0.8504 - val_loss: 0.3494
Epoch 2/3
352/352 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8785 - loss: 0.2854 - val_accuracy: 0.8416 - val_loss: 0.3712
Epoch 3/3
352/352 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.9075 - loss: 0.2343 - val_accuracy: 0.8264 - val_loss: 0.4356
[Encoder] Training done in 26.60s
[Encoder] Extracting features for SNN...
[Features] Shape: (25000, 256)

[SNN] Initializing 1SADP (T=25)...
[SNN] Training for 5 epochs...


  Epoch 1: Train Acc = 0.9307


  Epoch 2: Train Acc = 0.9327


...

  Epoch 3: Train Acc = 0.9354


  Epoch 4: Train Acc = 0.9355


  Epoch 5: Train Acc = 0.9359
[SNN] Evaluating on Test Set...
RESULTS: ag_news | LSTM | 1SADP -> Test Acc: 0.9020
      [Running] Arch: 1SADP | T: 50

[Data] Loading ag_news...
[Data] ag_news ready. Train shape: (120000, 100), Classes: 4

[Encoder] Initializing LSTM...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


[Encoder] Pre-training LSTM on ag_news for 3 epochs...
Epoch 1/3
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 439s 257ms/step - accuracy: 0.2526 - loss: 1.3884 - val_accuracy: 0.2444 - val_loss: 1.3861
Epoch 2/3
 494/1688 ━━━━━━━━━━━━━━━━━━━━ 4:54 246ms/step - accuracy: 0.2519 - loss: 1.3822